In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [11]:
from os.path import join

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

from models import MeanModel, SimpleModel, StandardModel, PMFModel
from solver import Solver

In [3]:
ls data/ml-100k/

README        u.info        u1.test       u4.base       ua.test
allbut.pl*    u.item        u2.base       u4.test       ub.base
mku.sh*       u.occupation  u2.test       u5.base       ub.test
u.data        u.user        u3.base       u5.test
u.genre       u1.base       u3.test       ua.base


In [4]:
def loadMovieLens(path):
    X_train = []
    y_train = []
    with open(join(path, 'u1.base')) as f:
        for line in f:
            uId, iId, r, timestamp = line.strip().split()
            X_train.append((int(uId), int(iId)))
            y_train.append(int(r))
    X_train = np.array(X_train, dtype=np.int32)
    y_train = np.array(y_train, dtype=np.float32)
            
    X_test = []
    y_test = []
    with open(join(path, 'u1.test')) as f:
        for line in f:
            uId, iId, r, timestamp = line.strip().split()
            X_test.append((int(uId), int(iId)))
            y_test.append(int(r))
    X_test = np.array(X_test, dtype=np.int32)
    y_test = np.array(y_test, dtype=np.float32)
    
    # Change Xs to 0 based indices.
    X_train -= 1
    X_test -= 1
              
    return X_train, y_train, X_test, y_test

In [5]:
X_train, y_train, X_test, y_test = loadMovieLens('data/ml-100k/')

In [16]:
yMean = np.mean(y_train)
y_train_centered = y_train - yMean
y_test_centered = y_test - yMean
centeredData = {
    'X_train': X_train,
    'y_train': y_train_centered,
    'X_val': X_test,
    'y_val': y_test_centered
}

In [6]:
nUsers = 1 + max(X_train[:, 0].max(), X_test[:, 0].max())
nItems = 1 + max(X_train[:, 1].max(), X_test[:, 1].max())
nUsers, nItems

(943, 1682)

In [7]:
data = {
    'X_train': X_train,
    'y_train': y_train,
    'X_val': X_test,
    'y_val': y_test
}

In [ ]:
model = SimpleModel(nUsers, nItems)
solver = Solver(model, data,
                update_rule='adam',
                optim_config={
                    'learning_rate': 1e-2,
                },
                lr_decay=0.9,
                num_epochs=10, batch_size=400,
                print_every=100)

In [ ]:
solver.train()

In [ ]:
plt.plot(solver.loss_history, 'o', alpha=0.7)
plt.ylim((0, 2))

In [ ]:
def plotTrainTest(solver):
    plt.plot(solver.train_loss_history, label='train')
    plt.plot(solver.val_loss_history, label='val')
    plt.legend()
    plt.xlabel('epoch')
    plt.ylabel('MSE')
    
plotTrainTest(solver)
plt.ylim((0, 2))

In [ ]:
mse, _ = model.loss(X_test, y_test, use_reg=False)
mse

In [ ]:
model2 = MeanModel()

In [ ]:
model2.fit(y_train)

In [ ]:
mse, _ = model2.loss(X_test, y_test)
mse

In [ ]:
model3 = StandardModel(nUsers, nItems, latentDim=10, reg=1e-3)
for p, v in model.params.items():
    model3.params[p] = v.copy()
    
solver3 = Solver(model3, data,
                update_rule='adam',
                optim_config={
                    'learning_rate': 3e-4,
                },
                lr_decay=1,
                num_epochs=10, batch_size=1000,
                print_every=100)
solver3.train()

In [ ]:
plt.plot(solver2.loss_history, 'o', alpha=0.7)

In [ ]:
plotTrainTest(solver3)

In [ ]:
mse, _ = model3.loss(X_test, y_test, use_reg=False)
mse

In [66]:
pmfModel = PMFModel(nUsers, nItems, lamU=.0001, lamV=.0001, latentDim=30)
pmfSolver = Solver(pmfModel, centeredData,
                update_rule='adam',
                optim_config={
                    'learning_rate': 3e-4,
                },
                lr_decay=.9,
                num_epochs=10, batch_size=400,
                print_every=100)

In [67]:
pmfSolver.train()

(Iteration 1 / 2000) loss: 1.178579
(Epoch 0 / 10) train loss: 1.240352; val_loss: 1.330968
(Iteration 101 / 2000) loss: 1.117180
(Epoch 1 / 10) train loss: 1.237032; val_loss: 1.327766
(Iteration 201 / 2000) loss: 1.256155
(Iteration 301 / 2000) loss: 1.254508
(Epoch 2 / 10) train loss: 1.227262; val_loss: 1.303422
(Iteration 401 / 2000) loss: 1.158193
(Iteration 501 / 2000) loss: 1.074126
(Epoch 3 / 10) train loss: 1.164066; val_loss: 1.265941
(Iteration 601 / 2000) loss: 1.213657
(Iteration 701 / 2000) loss: 1.234541
(Epoch 4 / 10) train loss: 1.151619; val_loss: 1.227856
(Iteration 801 / 2000) loss: 1.106188
(Iteration 901 / 2000) loss: 1.094234
(Epoch 5 / 10) train loss: 1.095121; val_loss: 1.195967
(Iteration 1001 / 2000) loss: 1.149971
(Iteration 1101 / 2000) loss: 0.947028
(Epoch 6 / 10) train loss: 1.078799; val_loss: 1.168987
(Iteration 1201 / 2000) loss: 1.134622
(Iteration 1301 / 2000) loss: 1.212550
(Epoch 7 / 10) train loss: 1.062387; val_loss: 1.147766
(Iteration 1401 / 

In [68]:
mse, _ = pmfModel.loss(X_test, y_test_centered, use_reg=False)
rmse = math.sqrt(mse)
rmse

1.0515923101272422

In [78]:
preds = np.dot(pmfModel.params['U'], pmfModel.params['V'].T)
preds = preds + yMean
np.sum(preds < 1.0)

0